## Creating the GPT-bot

Enter your own OpenAI api key if you want to run this notebook

In [1]:
import os
os.environ['OPENAI_API_KEY'] = "sk-..."   # Enter OpenAI API Key

In order to load our own PDF file to be able to ask questions from it, we fist need to read the pdf file and get all text from it. To do something like this, we can use a data loader from llama_index which easily lets us import all texts from a pdf file. 

In our case, the PDF file is going to be "A Brief Guide to Superheroes_ From Superman to the Avengers, the Evolution of Comic Book Legends.pdf" which talks about superheroes and how they have been represented in various domains of pop culture. The Bot is going to be capable of answering questions and framing it's own answers as long as the questions are answerable when the pdf is read.

In [3]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")
loader = PDFReader()
documents = loader.load_data(file=Path('./data/A Brief Guide to Superheroes_ From Superman to the Avengers, the Evolution of Comic Book Legends.pdf'))

In [4]:
# loading libraries and modules necessary to build our bot
from tqdm import tqdm
from pprint import pprint

import faiss
from keybert import KeyBERT
from transformers import AutoTokenizer
from llama_index import LangchainEmbedding, GPTFaissIndex, Document
from langchain.text_splitter import TextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In our case, we are going to be using embedding models from the excellent sentence-transformers library to embed all our texts into vectors. By default, llama_index uses GPT-3 embeddings. But this can be costly if the number of texts to embed are huge. 

So, in order to save costs and maintain the same quality of embeddings (in some cases, sentence-transformers models can perform even better), we use the 'multi-qa-mpnet-base-dot-v1' model which has been fine-tuned for semantic search: Given a query/question, if can find relevant passages. It was trained on a large and diverse set of (question, answer) pairs. So, it definitely is a good model to use for embedding all the texts with 0 extra cost.

Other than that, we also use a keyword extraction module called KeyBERT which can extract keywords by itself when given a sentence. This can be an optional parameter and is upto the user whether he wants to filter all retrieved texts initially by keywords before using GPT-3 to synthesize the final answer.

In [5]:
kw_model = KeyBERT()

embed_model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'  # d=768

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name=embed_model_name)) 
tokenizer = AutoTokenizer.from_pretrained(embed_model_name)

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=20)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/multi-qa-mpnet-base-dot-v1
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


We set up a FAISS index first which will store all the embedded vectors. FAISS is a great library for performing Aproximate Nearest Neighbour (ANN) search which helps us find the top-k most similar texts given a query text. There are multiple search algorithms implemented in this library which we can use for our work. We also set up a GPT Faiss Index into which we will push all our extracted documents (texts) from the PDF file. When we insert a document (text) into the GPT Faiss Index, it will automatically split up the text into further chunks, embed the texts into vectors and store them into the faiss index.

In [6]:
faiss_index = faiss.IndexHNSWFlat(768, 32)            
faiss_index.hnsw.efConstruction = 40         # Setting the value for efConstruction.
faiss_index.hnsw.efSearch = 16               # Setting the value for efSearch.

index = GPTFaissIndex(documents=documents, 
                      faiss_index=faiss_index, 
                      embed_model=embed_model,
                      use_async=False,
                     text_splitter=text_splitter)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 147894 tokens


We can now save the created faiss index locally so that we can re-use the index again without having to do the processing over the pdf file again.

In [7]:
# Saving the index files
index.save_to_disk(save_path='./indexes/index_list_emb.json', 
                   faiss_index_save_path='./indexes/faiss.index')

# # Loading the saved files
# GPTFaissIndex.load_from_disk(save_path='./indexes/index_list_emb.json',
#                              faiss_index_save_path='./indexes/faiss.index')

And that's essentially it! We can now use our created index to ask any questions about superheroes. To test it out, we create a function which when given a query/question, will look for the top k (default is 3) most similar texts in the index, and then synthesize and form a readable answer from the texts.

In [13]:
def get_answer(query: str, use_keyword_match: bool = False, similarity_top_k: int = 3):
    keyword = [kw_model.extract_keywords(query)[0][0]] if use_keyword_match else None
    response = index.query(
        query,
        mode="embedding",
        embed_model=embed_model,
        similarity_top_k=similarity_top_k,
        required_keywords=keyword
    )
    
    ids, sources = set(), []
    for source_node in response.source_nodes:
        sources.append(source_node.source_text)
    result = {"answer": response.response.strip(), "sources": sources}
    return result

We use the function to ask a question and see what answer it comes up with!

In [16]:
query = "Who is the alter-ego of Clark Kent?"
result = get_answer(query, use_keyword_match=False)

print(result['answer'])

The alter-ego of Clark Kent is Superman, a superhero with superhuman strength, speed, and other abilities. He is one of the most iconic and recognizable superheroes in the world, and has been featured in comics, television, and film. Superman is often seen as the archetypal superhero, and has been an inspiration for many other superheroes, such as Robin, the Boy Wonder, Rocket Raccoon, and Shazam.


And it works great, it directly understood what we asked and gave us the answer we wanted. All by itself! And the answers are pretty well-framed as well.

Moving forward, this is just like a question anwering bot, but we want the bot to have some personality of it's own! In comes langchain! It helps us connect our question answering method to a chatbot kind of module. So, we can interact with it in a more human-like manner. The following code cells perform this particular opertaion.

In [17]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.agents import initialize_agent

In [18]:
tools = [
    Tool(
        name = "CapedConversations",
        func=lambda q: str(index.query(q, 
                                       mode="embedding",
                                       embed_model=embed_model,
                                       similarity_top_k=3)),
        description="useful for when you want to answer questions about fictional superheroes. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]

In [19]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory)

Now that we are done with everything, let's test it out. I'll inform it my name and later ask the bot my name and see if it remebers what I said. In the middle, we can even ask questions about superheroes which we initially created and is now conencted to the chatbot.

In [20]:
agent_chain.run(input="Hi. This is Adirtha. What's up?")

'Hi Adirtha, how can I help you?'

In [23]:
agent_chain.run(input="How is Superman affected by Kryptonite?").strip()

'Superman is affected by Kryptonite in various ways, depending on the dose. It can lead to paralysis, loss of consciousness, or general weakness. In some cases, it can even be deadly. Exposure to Kryptonite can also cause Superman to lose his powers, making him vulnerable to his enemies such as Lex Luthor and the Kree. This has been seen in the past, such as when Lex Luthor used Element Z to create a Kryptonite ray to kill Superman in 1961, or when Superman was temporarily weakened by a cyborg pretender in 1992.'

In [24]:
agent_chain.run(input="What is my name?")

'Your name is Adirtha. Is there anything else I can help you with?'